<img style="float: right;" src="https://drive.google.com/uc?export=view&id=1qVA-sPHp5TY5D7enbU7mF4zZ8obu5wrC" width=150 height=150 />

### The Community Water Model (CWatM) is an open-source hydrological model developed at IIASA
This notebook runs CWatM using the excel file cwatm_settings.xlsx. Presented thereafter are outputs of the entire system as well as several highlighted components.
<br><br>

The **output_folder** in the accompanied notebook should point to where the simulations are held. <br>
The **basepath** should point to the folder "CWatM_settings" containing the two files:
 - *cwatm_settings.xlsx*: 
 - *settings_CwatM_template.ini*
 
 

In [7]:
#basepath = 'C:\GitHub\FUSE\CWatM_settings'
output_folder = 'C:/CWatM_output/Bhima'
#output_folder =r'C:\GitHub\bhima_repo\modules\hydro\hydro_outputs'
# cwatm_source_path must also be changed below

#basepath = r'C:\GitHub\bhima_repo_clean\modules\hydro\hydro_files\settings'
basepath = r'C:\GitHub\FUSE\CWatM_settings'

water_balance = True # most outputs
outputsForNotebooks = True # for discharge and waterbodies. A small subset of water_balance for discharge and waterbodies
AgricultureNotebook = True # For the LandCoverUse_CWatM notebook.
#AgInputsNotebook = True #TODO: For the AgInputs notebook
print_settings = False # Print the CWatM settings here



In [4]:
def create_cwatm_settings_dict_obj_TEST(xl_settings_file_path):
    import xlrd
    import pandas as pd
    # Create dictionary from excel file
    settings_dict = {}
    df = pd.read_excel(xl_settings_file_path, engine='openpyxl')
    wb = xlrd.open_workbook(xl_settings_file_path)
    sh = wb.sheet_by_index(0)
    for i in range(len(df)+1): #MS
        cell_key = sh.cell(i, 0).value
        cell_value = sh.cell(i, 1).value
        settings_dict[cell_key] = cell_value
    return settings_dict

    
def define_cwatm_settings_TEST(settings_template, new_settings_file, settings_dictionary):
    global Modflow
    sourcefile = open(settings_template, "r")
    newfile = open(new_settings_file, "wt")
    keylist = list(settings_dictionary.keys())
    for key in keylist:
        if 'modflow_coupling' in key:
            Modflow = settings_dictionary[key]
            Modflow_found = True
        
    for line in sourcefile:

        boolcheck = [elem in line for elem in keylist]
        match = any(elem in line for elem in keylist)
        ix = boolcheck.index(match)
        if 'modflow_coupling' in line and not 'Modflow_found' in locals():
            Modflow = False if line.split()[-1]==('False' or 'FALSE') else True
        if match:
            
            key = keylist.pop(ix)
            value = settings_dictionary[key]
            
            water_balance_outputs=''
            if key == 'OUT_MAP_Daily':
                if water_balance:
                    #water_balance_outputs= 'pumping_actual, gwstore, returnFlow, act_SurfaceWaterAbstract,act_nonIrrConsumption,sum_runoff,sum_interceptEvap,sum_actBareSoilEvap,act_totalIrrConsumption, sum_gwRecharge, gwstore, capillar, baseflow, act_bigLakeResAbst, leakage, actTransTotal_nonpaddy, actTransTotal_paddy, actTransTotal_grasslands, actTransTotal_forest, pumping_actual, pumping, cellArea, totalET, Rain, EvapoChannel, EvapWaterBodyM,  channelStorage, lakeResStorage, totalSto, storGroundwater, gwstore, sum_actTransTotal, sum_actBareSoilEvap, sum_interceptEvap, sum_openWaterEvap, addtoevapotrans,   lakeResInflowDis, lakeResOutflowDis, lakeResOutflowM, lakeResInflowM, discharge'
                    water_balance_outputs= 'act_indWithdrawal, act_domWithdrawal, act_livWithdrawal, act_indConsumption, act_domConsumption, act_livConsumption, lakeResInflowDis, lakeResOutflowDis, head_development, act_channelAbstract_Lift, act_nonLocalchannelAbstract_M, act_nonLocalLakeAbstract_M, sum_prefFlow_GW, sum_perc3toGW_GW, leakageIntoGw, leakageIntoRunoff,  riverbedExchangeM, act_bigLakeResAbst_alloc, act_LocalLakeAbstract, act_channelAbstract_Local, act_channelAbstract, act_irrWithdrawalGW, act_irrWithdrawalSW, act_nonIrrWithdrawalGW, act_nonIrrWithdrawalSW, act_nonIrrWithdrawal, act_irrWithdrawal, delivered_water, act_LocalLakeAbstract, resStorageTotal_alloc, pumping_actual, gwstore, returnFlow, act_SurfaceWaterAbstract,act_nonIrrConsumption,sum_runoff,sum_interceptEvap,sum_actBareSoilEvap,act_totalIrrConsumption, sum_gwRecharge, gwstore, capillar, baseflow, act_bigLakeResAbst, act_bigLakeResAbst_alloc, leakage, actTransTotal_nonpaddy, actTransTotal_paddy, actTransTotal_grasslands, actTransTotal_forest, pumping_actual, pumping, cellArea, totalET, Rain, EvapoChannel, EvapWaterBodyM,  channelStorage, lakeResStorage, totalSto, storGroundwater, gwstore, sum_actTransTotal, sum_actBareSoilEvap, sum_interceptEvap, sum_openWaterEvap, addtoevapotrans, lakeResOutflowM, lakeResInflowM, discharge'

                elif outputsForNotebooks:
                    water_balance_outputs= 'discharge, lakeResInflowDis, lakeResOutflowDis, lakeResStorage, act_bigLakeResAbst_alloc, act_LocalLakeAbstract, resStorageTotal_alloc'
        
                    
                else:
                    water_balance_outputs='discharge'
                if str(value)!='':
                    water_balance_outputs += ','
                
            if key == 'OUT_MAP_MonthEnd':
                if AgricultureNotebook == True:
                    water_balance_outputs= 'head_development, head_development_segments, modflowWaterLevel, fracVegCover[0], fracVegCover[1], fracVegCover[2], fracVegCover[3], fracVegCover[4], fracVegCover[5], frac_totalIrr, frac_totalnonIrr, fracCrops_Irr[0],  fracCrops_Irr[1], fracCrops_Irr[2], fracCrops_Irr[3], fracCrops_Irr[4], fracCrops_Irr[5], fracCrops_Irr[6], fracCrops_Irr[7], fracCrops_Irr[8], fracCrops_Irr[9], fracCrops_Irr[10], fracCrops_Irr[11], fracCrops_Irr[12], fracCrops_Irr[13], fracCrops_Irr[14], fracCrops_Irr[15], fracCrops_Irr[16], fracCrops_Irr[17], fracCrops_nonIrr[0], fracCrops_nonIrr[1], fracCrops_nonIrr[2], fracCrops_nonIrr[3], fracCrops_nonIrr[4], fracCrops_nonIrr[5], fracCrops_nonIrr[6], fracCrops_nonIrr[7],  fracCrops_nonIrr[8], fracCrops_nonIrr[9], fracCrops_nonIrr[10], fracCrops_nonIrr[11], fracCrops_nonIrr[12], fracCrops_nonIrr[13], fracCrops_nonIrr[14], fracCrops_nonIrr[15], fracCrops_nonIrr[16], fracCrops_nonIrr[17], fracCrops_IrrLandDemand[0],  fracCrops_IrrLandDemand[1], fracCrops_IrrLandDemand[2], fracCrops_IrrLandDemand[3], fracCrops_IrrLandDemand[4], fracCrops_IrrLandDemand[5], fracCrops_IrrLandDemand[6], fracCrops_IrrLandDemand[7],fracCrops_IrrLandDemand[8], fracCrops_IrrLandDemand[9], fracCrops_IrrLandDemand[10], fracCrops_IrrLandDemand[11], fracCrops_IrrLandDemand[12], fracCrops_IrrLandDemand[13], fracCrops_IrrLandDemand[14], fracCrops_IrrLandDemand[15], fracCrops_IrrLandDemand[16], fracCrops_IrrLandDemand[17], fracCrops_nonIrrLandDemand[0],  fracCrops_nonIrrLandDemand[1], fracCrops_nonIrrLandDemand[2], fracCrops_nonIrrLandDemand[3], fracCrops_nonIrrLandDemand[4], fracCrops_nonIrrLandDemand[5], fracCrops_nonIrrLandDemand[6], fracCrops_nonIrrLandDemand[7], fracCrops_nonIrrLandDemand[8], fracCrops_nonIrrLandDemand[9], fracCrops_nonIrrLandDemand[10], fracCrops_nonIrrLandDemand[11], fracCrops_nonIrrLandDemand[12], fracCrops_nonIrrLandDemand[13], fracCrops_nonIrrLandDemand[14], fracCrops_nonIrrLandDemand[15], fracCrops_nonIrrLandDemand[16], fracCrops_nonIrrLandDemand[17], fallowIrr, fallownonIrr, GeneralCrop_Irr, GeneralCrop_nonIrr, head_development, areaPaddy_Irr_segment, areaCrops_Irr_segment[0], areaCrops_nonIrr_segment[0], areaCrops_Irr_segment[1], areaCrops_nonIrr_segment[1], areaCrops_Irr_segment[2], areaCrops_nonIrr_segment[2], areaCrops_Irr_segment[3], areaCrops_nonIrr_segment[3], areaCrops_Irr_segment[4], areaCrops_nonIrr_segment[4], areaCrops_Irr_segment[5], areaCrops_nonIrr_segment[5], areaCrops_Irr_segment[6], areaCrops_nonIrr_segment[6], areaCrops_Irr_segment[7], areaCrops_nonIrr_segment[7], areaCrops_Irr_segment[8], areaCrops_nonIrr_segment[8], areaCrops_Irr_segment[9], areaCrops_nonIrr_segment[9], areaCrops_Irr_segment[10], areaCrops_nonIrr_segment[10], areaCrops_Irr_segment[11], areaCrops_nonIrr_segment[11], areaCrops_Irr_segment[12], areaCrops_nonIrr_segment[12], areaCrops_Irr_segment[13], areaCrops_nonIrr_segment[13], areaCrops_Irr_segment[14], areaCrops_nonIrr_segment[14], areaCrops_Irr_segment[15], areaCrops_nonIrr_segment[15] , areaCrops_Irr_segment[16], areaCrops_nonIrr_segment[16],  areaCrops_Irr_segment[17], areaCrops_nonIrr_segment[17], availableArableLand, availableArableLand_segment'
                    if str(value)!='':
                        water_balance_outputs += ','
                        
            newstring = key + ' = ' + water_balance_outputs + str(value) + '\n'
            newfile.write(newstring)
            
        else:
            newfile.write(line)
    sourcefile.close()
    newfile.close()

def show_cwatm_settings_file_TEST(settings_file_path):
    file = open(settings_file_path, "r")
    for line in file:
        print(line)
    file.close()

def run_cwatm_TEST(settings_file_path):
    import  os
    cwatm_source_path = 'C:\GitHub\CWATM_priv_FUSE\source_py3\cwatm3.py'
    #cwatm_source_path = r'C:\GitHub\bhima_repo_clean\python_packages\source_py3\cwatm3.py'
    command_str = 'python ' + cwatm_source_path + " " + settings_file_path + ' -l'
    flag = os.system(command_str)
    if flag == 0:
        print('Successfully Finished Running CWATM\n')

In [5]:
import time
from datetime import date

today = date.today().strftime("%B%d")

settings_template = basepath + r'\settings_CWatM_template.ini'
excel_settings_file =basepath + r'\cwatm_settings.xlsx'
#excel_settings_file =basepath + r'\cwatm_settings_12Oct.xlsx'
new_settings_file = basepath + r'\settings_CWatM_'+today+'.ini'

# Create an initial settings dictionary object from the excel file.
# This dictionary be modified by the urban and ag calculations to pass file paths or values
settings_dict = create_cwatm_settings_dict_obj_TEST(excel_settings_file)

define_cwatm_settings_TEST(settings_template, new_settings_file, settings_dict)

if print_settings == True:
    show_cwatm_settings_file_TEST(new_settings_file)
    

run_cwatm_TEST(new_settings_file)

AttributeError: 'ElementTree' object has no attribute 'getiterator'

In [9]:
run ./1_Balance_CWatM.ipynb

RuntimeError: NetCDF: HDF error

RuntimeError: NetCDF: HDF error